# Introduction to Docker

**Learning Objectives**
  * Build and run Docker containers
  * Pull Docker images from Docker Hub and Google Container Registry
  * Push Docker images to Google Container Registry

## Overview

Docker is an open platform for developing, shipping, and running applications. With Docker, you can separate your applications from your infrastructure and treat your infrastructure like a managed application. Docker helps you ship code faster, test faster, deploy faster, and shorten the cycle between writing code and running code.

Docker does this by combining kernel containerization features with workflows and tooling that helps you manage and deploy your applications.

Docker containers can be directly used in Kubernetes, which allows them to be run in the Kubernetes Engine with ease. After learning the essentials of Docker, you will have the skillset to start developing Kubernetes and containerized applications.

## Basic Docker commands

See what docker images you have. 

In [1]:
!docker images

REPOSITORY   TAG       IMAGE ID   CREATED   SIZE


If this is the first time working with docker you won't have any repositories listed. 

**Note**. If you are running this in an AI Notebook, then you should see a single image `gcr.io/inverting-proxy/agent`. This is the container that is currently running the AI Notebook. 

Let's use `docker run` to pull a docker image called `hello-world` from the public registry. The docker daemon will search for the `hello-world` image, if it doesn't find the image locally, it pulls the image from a public registry called Docker Hub, creates a container from that image, and runs the container for you.

In [2]:
!docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

85e32844: Pull complete 457kB/2.457kBBDigest: sha256:926fac19d22aa2d60f1a276b66a20eb765fbeea2db5dbdaafeb456ad8ce81598
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For 

Now when we look at our docker images we should see `hello-world` there as well.

In [3]:
!docker images

REPOSITORY    TAG       IMAGE ID       CREATED        SIZE
hello-world   latest    9c7a54a9a43c   2 months ago   13.3kB


This is the image pulled from the Docker Hub public registry. The Image ID is in `SHA256` hash format—this field specifies the Docker image that's been provisioned. When the docker daemon can't find an image locally, it will by default search the public registry for the image. Let's run the container again:

Now, if we want to run `docker run hello-world` again, it won't have to download from the container registry.

To see all docker containers running, use `docker ps`.

In [4]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


There are no running containers. **Note. If you are running this in at AI Notebook, you'll see one container running.**

The `hello-world` containers you ran previously already exited. In order to see all containers, including ones that have finished executing, run docker `ps -a`:

In [5]:
!docker ps -a

CONTAINER ID   IMAGE         COMMAND    CREATED          STATUS                      PORTS     NAMES
9120d8443e2a   hello-world   "/hello"   36 seconds ago   Exited (0) 35 seconds ago             happy_boyd


This shows you the Container ID, a UUID generated by Docker to identify the container, and more metadata about the run. The container Names are also randomly generated but can be specified with docker run --name [container-name] hello-world.

## Build a Docker container

Let's build a Docker image that's based on a simple node application. 

Open a new text file and write the following. Save the file in a folder called `dockerfiles` and name the file `intro.docker`

```bash
# Use an official Node runtime as the parent image
FROM node:6

# Set the working directory in the container to /app
WORKDIR /app

# Copy the current directory contents into the container at /app
ADD . /app

# Make the container's port 80 available to the outside world
EXPOSE 80

# Run app.js using node when the container launches
CMD ["node", "./src/app.js"]
```

This file instructs the Docker daemon on how to build your image.

The initial line specifies the base parent image, which in this case is the official Docker image for node version 6.
In the second, we set the working (current) directory of the container.
In the third, we add the current directory's contents (indicated by the "." ) into the container.
Then we expose the container's port so it can accept connections on that port and finally run the node command to start the application.

Check out the other [Docker command references](https://docs.docker.com/engine/reference/builder/#known-issues-run) to understand what each line does.

We're going to use this Docker container to run a simple node.js app. Have a look at `app.js`. This is a simple HTTP server that listens on port 80 and returns "Hello World."


Now let's build the image. Note again the "`.`", which means current directory so you need to run this command from within the directory that has the Dockerfile.

The `-t` is to name and tag an image with the `name:tag` syntax. The name of the image is `node-app` and the tag is `0.1`. The tag is highly recommended when building Docker images. If you don't specify a tag, the tag will default to latest and it becomes more difficult to distinguish newer images from older ones. Also notice how each line in the Dockerfile above results in intermediate container layers as the image is built.

In [6]:
!docker build -t node-app:0.1 -f dockerfiles/intro.docker .

Sending build context to Docker daemon  77.82kB
Step 1/5 : FROM node:6
6: Pulling from library/node

55d5a1d1: Pulling fs layer 
80d00ae9: Pulling fs layer 
b3117dca: Pulling fs layer 
a19181b2: Pulling fs layer 
7b2a5bcc: Pulling fs layer 
12c70287: Pulling fs layer 
5386a42d: Pulling fs layer 
Digest: sha256:e133e66ec3bfc98da0440e552f452e5cdf6413319d27a2db3b01ac4b319759b38A
Status: Downloaded newer image for node:6
 ---> ab290b853066
Step 2/5 : WORKDIR /app
 ---> Running in ae99ca5c283a
Removing intermediate container ae99ca5c283a
 ---> 483a0fbcaabe
Step 3/5 : ADD . /app
 ---> 0d7a5c7adf5d
Step 4/5 : EXPOSE 80
 ---> Running in 5b157934df19
Removing intermediate container 5b157934df19
 ---> 5f28099307b5
Step 5/5 : CMD ["node", "./src/app.js"]
 ---> Running in dc9f66c51a6b
Removing intermediate container dc9f66c51a6b
 ---> be5824534ab2
Successfully built be5824534ab2
Successfully tagged node-app:0.1


Let's check that the image has been created correctly. 

In [7]:
!docker images

REPOSITORY    TAG       IMAGE ID       CREATED                  SIZE
node-app      0.1       be5824534ab2   Less than a second ago   884MB
hello-world   latest    9c7a54a9a43c   2 months ago             13.3kB
node          6         ab290b853066   4 years ago              884MB


You should see a `node-app` repository that was created only seconds ago. 

Notice `node` is the base image and `node-app` is the image you built. You can't remove `node` without removing `node-app` first. The size of the image is relatively small compared to VMs. Other versions of the node image such as `node:slim` and `node:alpine` can give you even smaller images for easier portability. The topic of slimming down container sizes is further explored in Advanced Topics. You can view all versions in the official repository here.

Note, you can remove an image from your docker images using `docker rmi [repository]:[tag]`.

## Run a Docker container

Now we'll run the container based on the image you built above using the `docker run` command. The `--name` flag allows you to name the container if you like. And `-p` instructs Docker to map the host's port 4000 to the container's port 80. This allows you to reach the server at http://localhost:4000. Without port mapping, you would not be able to reach the container at localhost.

In [8]:
!docker ps -a

CONTAINER ID   IMAGE         COMMAND    CREATED         STATUS                     PORTS     NAMES
9120d8443e2a   hello-world   "/hello"   2 minutes ago   Exited (0) 2 minutes ago             happy_boyd


In [9]:
!docker run -p 4000:80 --name my-app node-app:0.1

Server running at http://0.0.0.0:80/


To test out the server, open a terminal window and type the following command:

```bash
curl http://localhost:4000
```

You should see the server respond with `Hello World`

The container will run as long as the initial terminal is running. If you want to stop the container, run the following command in the terminal to stop and remove the container:

```bash
docker stop my-app && docker rm my-app
```
After a few moments the container will stop. You should notice the cell above will complete execution.

#### Running the container in the background
If you want to the container to run in the background (not tied to the terminal's session), you need to specify the `-d` flag.
Now run the following command to start the container in the background

In [10]:
!docker run -p 4000:80 --name my-app -d node-app:0.1

fd104cff80e9652c990de82a90d915fdf5f42f0eaea4c819438aa045427c3bb1


Your container is now running in the background. You can check the status of your running container using `docker ps`

In [11]:
!docker ps

CONTAINER ID   IMAGE          COMMAND               CREATED         STATUS         PORTS                                   NAMES
fd104cff80e9   node-app:0.1   "node ./src/app.js"   4 seconds ago   Up 3 seconds   0.0.0.0:4000->80/tcp, :::4000->80/tcp   my-app


Notice the container is running in the output of docker ps. You can look at the logs by executing `docker logs [container_id]`. 

In [13]:
# Note, your container id will be different
!docker logs fd104cff80e9

Server running at http://0.0.0.0:80/


You should see 
```bash
Server running at http://0.0.0.0:80/
```
If you want to follow the log's output as the container is running, use the `-f` option.

## Modify & Publish

Let's modify the application and push it to your Google Cloud Repository (gcr). After that you'll remove all local containers and images to simulate a fresh environment, and then pull and run your containers from gcr. This will demonstrate the portability of Docker containers.

### Edit `app.js`
Open the file `./src/app.js` with the text editor and replace "Hello World" with another string. Then build this new image. 

In [14]:
!docker build -t node-app:0.2 -f dockerfiles/intro.docker .

Sending build context to Docker daemon  87.55kB
Step 1/5 : FROM node:6
 ---> ab290b853066
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> 483a0fbcaabe
Step 3/5 : ADD . /app
 ---> 64fe00922057
Step 4/5 : EXPOSE 80
 ---> Running in 38f96deb397c
Removing intermediate container 38f96deb397c
 ---> 289bb427581c
Step 5/5 : CMD ["node", "./src/app.js"]
 ---> Running in 7f92e5530467
Removing intermediate container 7f92e5530467
 ---> a4a686840c62
Successfully built a4a686840c62
Successfully tagged node-app:0.2


Notice in `Step 2` of the output we are using an existing cache layer. From `Step 3` and on, the layers are modified because we made a change in `app.js`.

Run another container with the new image version. Notice how we map the host's port 8000 instead of 80. We can't use host port 4000 because it's already in use. 

In [15]:
!docker run -p 8000:80 --name my-app-2 -d node-app:0.2

18588967d273f43ee1014454744d42a460d574255bcc46e2769f5f24d99e5fb5


You can check that both container are running using `docker ps`.

In [16]:
!docker ps

CONTAINER ID   IMAGE          COMMAND               CREATED              STATUS              PORTS                                   NAMES
18588967d273   node-app:0.2   "node ./src/app.js"   3 seconds ago        Up 2 seconds        0.0.0.0:8000->80/tcp, :::8000->80/tcp   my-app-2
fd104cff80e9   node-app:0.1   "node ./src/app.js"   About a minute ago   Up About a minute   0.0.0.0:4000->80/tcp, :::4000->80/tcp   my-app


And let's test boht containers using `curl` as before:

In [17]:
!curl http://localhost:8000

Hello Beautiful World!!


In [19]:
!curl http://localhost:4000

Hello World!!


Recall, to stop a container running, you can execute the following command either in a terminal or (because they are running in the background) in a cell in this notebook. 

### Publish to gcr

Now you're going to push your image to the Google Container Registry (gcr). To push images to your private registry hosted by gcr, you need to tag the images with a registry name. The format is `[hostname]/[project-id]/[image]:[tag]`.

For gcr:

  * `[hostname]`= gcr.io
  * `[project-id]`= your project's ID
  * `[image]`= your image name
  * `[tag]`= any string tag of your choice. If unspecified, it defaults to "latest".

In [20]:
import os

PROJECT_ID = "qwiklabs-asl-03-2de90311f4ec"  # REPLACE WITH YOUR PROJECT NAME
os.environ["PROJECT_ID"] = PROJECT_ID

Let's tag `node-app:0.2`.

In [21]:
!docker images

REPOSITORY    TAG       IMAGE ID       CREATED          SIZE
node-app      0.2       a4a686840c62   28 minutes ago   884MB
node-app      0.1       be5824534ab2   30 minutes ago   884MB
hello-world   latest    9c7a54a9a43c   2 months ago     13.3kB
node          6         ab290b853066   4 years ago      884MB


In [22]:
%%bash
docker tag node-app:0.2 gcr.io/${PROJECT_ID}/node-app:0.2

Now when we list our docker images we should see this newly tagged repository.

In [23]:
!docker images

REPOSITORY                                     TAG       IMAGE ID       CREATED          SIZE
node-app                                       0.2       a4a686840c62   28 minutes ago   884MB
gcr.io/qwiklabs-asl-03-2de90311f4ec/node-app   0.2       a4a686840c62   28 minutes ago   884MB
node-app                                       0.1       be5824534ab2   30 minutes ago   884MB
hello-world                                    latest    9c7a54a9a43c   2 months ago     13.3kB
node                                           6         ab290b853066   4 years ago      884MB


Next, let's push this image to gcr.

In [24]:
%%bash
docker push gcr.io/${PROJECT_ID}/node-app:0.2

The push refers to repository [gcr.io/qwiklabs-asl-03-2de90311f4ec/node-app]
327e3b55823b: Preparing
e115c18d19da: Preparing
f39151891503: Preparing
f1965d3c206f: Preparing
a27518e43e49: Preparing
910d7fd9e23e: Preparing
4230ff7f2288: Preparing
2c719774c1e1: Preparing
ec62f19bb3aa: Preparing
f94641f1fe1f: Preparing
2c719774c1e1: Waiting
ec62f19bb3aa: Waiting
910d7fd9e23e: Waiting
f94641f1fe1f: Waiting
4230ff7f2288: Waiting
a27518e43e49: Layer already exists
f39151891503: Layer already exists
f1965d3c206f: Layer already exists
4230ff7f2288: Layer already exists
2c719774c1e1: Layer already exists
910d7fd9e23e: Layer already exists
f94641f1fe1f: Layer already exists
ec62f19bb3aa: Layer already exists
e115c18d19da: Pushed
327e3b55823b: Pushed
0.2: digest: sha256:33b4d1bfc5cc86271bed005671f10e5de5b2f57f5a1885fb2767a3f7ff1312d2 size: 2424


Check that the image exists in `gcr` by visiting the image registry Cloud Console. You can navigate via the console to `Navigation menu > Container Registry` or visit the url from the cell below:

In [25]:
%%bash
echo "http://gcr.io/${PROJECT_ID}/node-app"

http://gcr.io/qwiklabs-asl-03-2de90311f4ec/node-app


### Test the published gcr image

Let's test this image. You could start a new VM, ssh into that VM, and install gcloud. For simplicity, we'll just remove all containers and images to simulate a fresh environment.

First, stop and remove all containers using `docker stop` and `docker rm`. **Be careful not to stop the container running this AI Notebook!**.

In [26]:
!docker stop my-app && docker rm my-app

my-app
my-app


In [35]:
!docker stop my-app-2 && docker rm my-app-2

Error response from daemon: No such container: my-app-2


Now remove the docker images you've created above using `docker rmi`.

In [28]:
!docker images

REPOSITORY                                     TAG       IMAGE ID       CREATED          SIZE
node-app                                       0.2       a4a686840c62   29 minutes ago   884MB
gcr.io/qwiklabs-asl-03-2de90311f4ec/node-app   0.2       a4a686840c62   29 minutes ago   884MB
node-app                                       0.1       be5824534ab2   31 minutes ago   884MB
hello-world                                    latest    9c7a54a9a43c   2 months ago     13.3kB
node                                           6         ab290b853066   4 years ago      884MB


In [34]:
%%bash
docker rmi node-app:0.2
docker rmi gcr.io/${PROJECT_ID}/node-app:0.2
docker rmi node-app:0.1
docker rmi node:6 
docker rmi -f hello-world:latest

Error: No such image: node-app:0.2
Error response from daemon: conflict: unable to remove repository reference "gcr.io/qwiklabs-asl-03-2de90311f4ec/node-app:0.2" (must force) - container c74a2026e761 is using its referenced image a4a686840c62
Error: No such image: node-app:0.1
Error: No such image: node:6
Error: No such image: hello-world:latest


Confirm all images are removed with `docker images`.

In [30]:
!docker images

REPOSITORY   TAG       IMAGE ID   CREATED   SIZE


At this point you should have a pseudo-fresh environment. Now, pull the image and run it.

In [31]:
%%bash
docker pull gcr.io/${PROJECT_ID}/node-app:0.2
docker run -p 4000:80 -d gcr.io/${PROJECT_ID}/node-app:0.2

0.2: Pulling from qwiklabs-asl-03-2de90311f4ec/node-app
c5e155d5a1d1: Pulling fs layer
221d80d00ae9: Pulling fs layer
4250b3117dca: Pulling fs layer
3b7ca19181b2: Pulling fs layer
425d7b2a5bcc: Pulling fs layer
69df12c70287: Pulling fs layer
ea2f5386a42d: Pulling fs layer
d421d2b3c5eb: Pulling fs layer
dc228a9530ce: Pulling fs layer
3b7ca19181b2: Waiting
f6a6efa857df: Pulling fs layer
425d7b2a5bcc: Waiting
69df12c70287: Waiting
d421d2b3c5eb: Waiting
ea2f5386a42d: Waiting
dc228a9530ce: Waiting
f6a6efa857df: Waiting
4250b3117dca: Verifying Checksum
4250b3117dca: Download complete
221d80d00ae9: Download complete
c5e155d5a1d1: Verifying Checksum
c5e155d5a1d1: Download complete
69df12c70287: Verifying Checksum
69df12c70287: Download complete
3b7ca19181b2: Verifying Checksum
3b7ca19181b2: Download complete
d421d2b3c5eb: Verifying Checksum
d421d2b3c5eb: Download complete
ea2f5386a42d: Verifying Checksum
ea2f5386a42d: Download complete
dc228a9530ce: Download complete
f6a6efa857df: Verifying Ch

You can check that it's running as expected using before:

In [33]:
!curl http://localhost:4000

Hello Beautiful World!!


Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.